In [119]:
import pandas as pd
import numpy as np
import haversine as hs
from haversine import Unit 
import random
import sys
import math
import warnings
warnings.filterwarnings("ignore")

In [91]:
#read in cleaned dataframe
df = pd.read_parquet("../data/df_cleaned_21_08_2021.parquet")

In [92]:
#reducing dataframe
def sampleFile(sample_size,df):
    df_new = df.sample(frac = (sample_size/100),random_state=42)
    return df_new

df_all= df.copy()
df = sampleFile(10,df)
print("New length of dataframe: ",len(df))

New length of dataframe:  108989


In [138]:
df_choosen_date = df[df['PU_DATE'].astype(str) == '2017-06-15']
df_sample_coordinates = df_choosen_date[['Pickup Centroid Latitude','Pickup Centroid Longitude']]
df_sample_coordinates.reset_index(drop = True, inplace = True)
df_sample_coordinates.rename(columns={"Pickup Centroid Latitude": "lat_i", "Pickup Centroid Longitude": "lon_i"},
                             inplace = True)
df_sample_coordinates.dropna(inplace=True)
print("Choosen df lenght: ",len(df_sample_coordinates))
#df_sample_coordinates['location driver'] = np.arange(len(df_sample_coordinates))
df_sample_coordinates = list(zip(df_sample_coordinates['lat_i'], df_sample_coordinates['lon_i']))
df_sample_coordinates

Choosen df lenght:  397


[(41.89206541810932, -87.61197123778544),
 (41.88269297991777, -87.6198973780236),
 (41.888943021728686, -87.63308398397615),
 (41.89206541810932, -87.61197123778544),
 (41.84562319331464, -87.61185096556288),
 (42.01567151884942, -87.68255206747911),
 (41.87790356810432, -87.63379557974066),
 (41.883317194090495, -87.61567540083753),
 (41.87956975737649, -87.6410048938619),
 (41.81582263132915, -87.73902635250049),
 (41.94769406235561, -87.67953563805462),
 (41.87894468266093, -87.64522592275242),
 (41.87498775719616, -87.63502863838762),
 (41.894775731759424, -87.63061670471049),
 (41.99396852880858, -87.69964392625343),
 (41.87456984822307, -87.61937812767056),
 (41.8872762166862, -87.62587335209787),
 (41.87561241613522, -87.63080772700704),
 (41.78668422339367, -87.75126373919485),
 (41.894566357709294, -87.65049606004496),
 (41.87977709308188, -87.62113148388708),
 (41.98106001883225, -87.65751460141291),
 (41.893525797631476, -87.63906188086226),
 (41.885608964340065, -87.618663

# Get data of places in Chicago

In [209]:
latitude =  41.881832
longitude = -87.623177
lat_list = []
long_list = []

def generate_random_data(lat, lon, num_rows):
    for _ in range(num_rows):
        hex1 = '%012x' % random.randrange(16**12) # 12 char random string
        flt = float(random.randint(0,100))
        dec_lat = random.random()/100
        dec_lon = random.random()/100
        long_list.append(lon+dec_lon)
        lat_list.append(lat+dec_lat)
    
    #df = pd.DataFrame({'lat_j':lat_list, 'lon_j':long_list})#
    matrix = list(zip(lat_list, long_list))
    #matrix = list(zip(matrix, long_list))
    return matrix

matrix_places = generate_random_data(latitude, longitude, 10)
matrix_places

[(41.88767957235624, -87.62256682079207),
 (41.88933318795759, -87.61429117795304),
 (41.883469946918886, -87.62133320197906),
 (41.881963918639855, -87.62311407681125),
 (41.88288178647935, -87.61339144309035),
 (41.886298825033755, -87.61369098747996),
 (41.88442324576388, -87.62275017498054),
 (41.89085708135197, -87.61849792540066),
 (41.886329221831886, -87.62138044604497),
 (41.88386929780747, -87.61731916815513)]

# Matrix with Haversine distance

In [210]:
df_sample_coordinates

[(41.89206541810932, -87.61197123778544),
 (41.88269297991777, -87.6198973780236),
 (41.888943021728686, -87.63308398397615),
 (41.89206541810932, -87.61197123778544),
 (41.84562319331464, -87.61185096556288),
 (42.01567151884942, -87.68255206747911),
 (41.87790356810432, -87.63379557974066),
 (41.883317194090495, -87.61567540083753),
 (41.87956975737649, -87.6410048938619),
 (41.81582263132915, -87.73902635250049),
 (41.94769406235561, -87.67953563805462),
 (41.87894468266093, -87.64522592275242),
 (41.87498775719616, -87.63502863838762),
 (41.894775731759424, -87.63061670471049),
 (41.99396852880858, -87.69964392625343),
 (41.87456984822307, -87.61937812767056),
 (41.8872762166862, -87.62587335209787),
 (41.87561241613522, -87.63080772700704),
 (41.78668422339367, -87.75126373919485),
 (41.894566357709294, -87.65049606004496),
 (41.87977709308188, -87.62113148388708),
 (41.98106001883225, -87.65751460141291),
 (41.893525797631476, -87.63906188086226),
 (41.885608964340065, -87.618663

In [191]:
def build_matrix(rows, cols):
    matrix = []

    for r in range(0, rows):
        matrix.append([0 for c in range(0, cols)])

    return matrix

In [226]:
distance_matrix = build_matrix(len(df_sample_coordinates),len(matrix_places))


for i in range(0,len(matrix_places)):
    for j in range(0,len(df_sample_coordinates)):
        gps_i = (matrix_places[i][0],matrix_places[i][1])
        gps_j = (df_sample_coordinates[j][0],df_sample_coordinates[j][1])
        
        
        distance_matrix[j][i] = hs.haversine(gps_j,gps_i,unit=Unit.MILES)

    
    


In [228]:
distance_matrix = np.reshape(distance_matrix, (10, 397)).T
distance_matrix

array([[ 0.6235692 ,  0.26256739,  1.00218304, ...,  0.60633132,
         1.14749785,  4.74360456],
       [ 4.83926105,  8.15922807,  1.21758072, ...,  3.77251669,
         0.73648913,  1.14913076],
       [ 1.21758072,  0.89507656, 10.19672199, ...,  0.03883678,
         2.11104749,  0.74761118],
       ...,
       [ 2.02428963,  0.96793219,  1.23955183, ...,  1.09724833,
         0.40122097,  1.71888097],
       [ 1.51139351,  0.26382394,  2.10829632, ...,  1.43510196,
         1.23955183,  4.42141732],
       [ 0.9090327 ,  0.69930866,  1.47535208, ...,  0.82638825,
         9.63005839,  0.81194956]])

# todo: calculate prices for distances of distance_matrix.


Einheit sind Miles. 397 Datenpunkte * 10 columns ()